In [41]:
from dotenv import dotenv_values
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings
# my twitter keys are stores in a file outside the git repository for security reasons
config = dotenv_values("/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_DS_CA2/.env")
bearer_token = config["BEARER_TOKEN"]
# this is the location of the twitter api access , looking for the most recent tweets
search_url = "https://api.twitter.com/2/tweets/search/recent"

# Set up and run API query. Save result to .csv

In [42]:


# Set up the query we want to pass to api , is a json object
    #   Im adding parameter -is = is not a retweet. i dont want the same retweets showing up in my data 
    # also pulling in the author id and name
# Im using two query_params the first is without the next_token field for the first query   
query_params1 = {
    'query':'BREXIT farmers -is:retweet',
    'max_results': '100',
    'tweet.fields':'author_id',
    'user.fields':'name',

}
# This second query is for the loop , as it requires a next_token.
query_params2 = {
    'query':'BREXIT farmers -is:retweet',
    'max_results': '100',
    'tweet.fields':'author_id',
    'user.fields':'name',
    'next_token' : 'abcd',
}

# my authorisation keys are used here for ther twitter API
def bearer_oauth(r):
    """ Function for using bearer token"""
    r.headers['Authorization'] = f"Bearer {bearer_token}"
    r.headers['User-Agent'] = "v2RecentSerchPython"
    return r


def connect_to_endpoint(url,params):
    """ Function to connect to twitter API"""
    response = requests.get(url,auth=bearer_oauth, params=params)
    #responce.status_code 200 is good anything else is an error
    
    return response.json()

def append_dict(adict,json_response):
    for i in range(len(json_response['data'])):
        try:
            adict[json_response['data'][i]['author_id']] = json_response['data'][i]['text']

        except IndexError:
            print("We had an index error")
    return(adict)

    
def twit_call():
    twit_dict={}
    loop_count = 0
    json_response = connect_to_endpoint(search_url, query_params1)

    append_dict(twit_dict,json_response)
    # while json_response['meta']['next_token']:
    while True:
        # pagenation is taking the next_token and feeding it through to next query
        if 'next_token' in json_response['meta']: 
            next_t = json_response['meta']['next_token']
            query_params2['next_token'] = next_t
        else:
            # this is the last page
            break
        json_response = connect_to_endpoint(search_url, query_params2)
        append_dict(twit_dict,json_response)
           
    return(twit_dict)

my_dict = twit_call()

In [43]:
# convert dictionary to dataframe
df = pd.DataFrame.from_dict(my_dict, orient='index',columns=['tweets'])

In [44]:
df.shape

(584, 1)

In [45]:
df.head()

,tweets
1030824102070808582,"POST-BREXIT £1,000 FARMING PAYMENTS ‘TOO LITTL..."
3366508409,@waltjack71 @sainsburys @Tesco @1GarethWynJone...
1275219963775778816,@waltjack71 @mikegalsworthy @sainsburys @Tesco...
48992513,"#BrexitReality Post-#Brexit £1,000 farming pay..."
1246437728679530496,More #BrexitLies \nScheme supposed to replace ...


In [46]:
# save the dataframe as cvs file.
#df.to_csv('BREXIT_tweets.csv')


# Load previously saved tweets

In [47]:
#df = pd.read_csv("BREXIT_tweets_5_jan.csv")

In [48]:
df.head()

,tweets
1030824102070808582,"POST-BREXIT £1,000 FARMING PAYMENTS ‘TOO LITTL..."
3366508409,@waltjack71 @sainsburys @Tesco @1GarethWynJone...
1275219963775778816,@waltjack71 @mikegalsworthy @sainsburys @Tesco...
48992513,"#BrexitReality Post-#Brexit £1,000 farming pay..."
1246437728679530496,More #BrexitLies \nScheme supposed to replace ...


In [49]:
import nltk

In [26]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ambrosedesmond/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# using regular expressions to remove all special charecters.
df["tweets"] = df['tweets'].str.replace('[^\w\s]','')

In [28]:
# using regular expressions to remove all tabs and carrige returns that were imported in original csv
df["tweets"] = df['tweets'].str.replace(r'\r\n','')

In [29]:
# Basically stop words like this, an, a, the, etc that do not affect the meaning of the tweet will be removed
from nltk.corpus import stopwords
stop = stopwords.words("english")

In [30]:
# lambda counts the  number of stop words in each tweet
df["stopwords"]  = df["tweets"].apply(lambda x : len([x for x in x.split() if x in stop]))

In [31]:
# lambda counts the number of upper case words in tweet , indicate shouting , anger ect
df["upper"]  = df["tweets"].apply(lambda x : len([x for x in x.split() if x.isupper()]))

In [32]:

#pip install textblob

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 583 entries, 3366508409 to 1282693762385403906
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweets     583 non-null    object
 1   stopwords  583 non-null    int64 
 2   upper      583 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 18.2+ KB


In [34]:
df['tweets'] = df['tweets'].astype(str)

In [ ]:
##################### Warning this cell can take up to 20 mins to complete ################### 
# corrects spellings and grammer based on context
# use txtblob to correct the tweet spelling
from textblob import TextBlob
df['tweets_correct?']= df["tweets"].apply(lambda x : str(TextBlob(x).correct()))
# pattern matching is not good

In [18]:
# get the sentiment . using scored words. scores are added up and the final score is the entimenen 
# The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the 
# range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
df['sentiment_1'] = df["tweets"].apply(lambda x : TextBlob(x).sentiment[0])
df['sentiment_2'] = df["tweets_correct?"].apply(lambda x : TextBlob(x).sentiment[0])

NameError: name 'TextBlob' is not defined

In [ ]:
df.head()

In [ ]:
df = df.sort_values(by='sentiment_1', ascending=False)
df = df.sort_values(by='sentiment_2', ascending=False)

In [ ]:
# the histogram gives a instant view of all sentiment , and changes from day to day
# depending on the news topic of the day.
df.hist(column='sentiment_1')

plt.show()

In [ ]:
# The histogram give a visual indication of the sentiment, 
# we see the majority of the sentiment is around 0 on the x axis indicating neutral sentiment 
# in relation to Brexit and farming on the day the reading was taken. Previous readings 
# have tended to be mostly negative.
df.hist(column='sentiment_2')

plt.show()

## BAG OF WORDS

In [ ]:
# Store the comments
X = df["tweets"]
y = df["sentiment_1"]

In [ ]:
y.head()

In [ ]:
# Lamdba to categorise the sentiment to 0,1,2 groupings
y = y.apply(lambda x: 0 if x < -0.5 else (1 if x < 0.5 else 2))

In [ ]:
y.head()

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

import string
from nltk.stem import PorterStemmer

In [ ]:
# Store the stopwords into the object named as "stop_words"
# Stopwords are words which do not contain enough significance to be used without our algorithm
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct. Removing Punctuation: ‘,.*!’ and other 
# punctuation marks that are not really needed by the model
punct = string.punctuation

# Initialise an object using a method PorterStemmer.
# reducing words like ‘jumping, jumped, jump’ into its root word(also called stem)
stemmer = PorterStemmer()

In [ ]:
import re

cleaned_data=[]

# For loop from first value to length(X), ^a-zA-Z means include small and capital case letters

for i in range(len(X)):
    message = re.sub('[^a-zA-Z]', ' ', X.iloc[i])
    message = message.lower().split()
    message = [stemmer.stem(word) for word in message if (word not in stop_words) and (word not in punct)]
    # we rejoin all the words back into a sentance again and append the sentances into a list
    message = ' '.join(message)
    cleaned_data.append(message)

In [ ]:
X # this is the regular cleaned data, note I have full sentices with all the unnecessary words

In [ ]:
# Unnecessary (stop words) removed but meaning still intact. the stemer sometimes abbreviates words too
cleaned_data 

In [ ]:
# Tokenization is the act of breaking up a sequence of strings into pieces such 
# as words, keywords, phrases, symbols and other elements called tokens
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate an object cv by calling a method named as CountVectorzer()
# here we are only taking 3K columns else the dataset will be too big
# one con here is the order of the words is lost so the meaning can be lost a little
# set stop words to names that dont indicate sentiment
cv    = CountVectorizer(max_features = 3000, stop_words = ['BREXIT'])

# convert the cleaneddata into the bag of words we want
X_fin = cv.fit_transform(cleaned_data).toarray()

# Display the rows and colums
X_fin.shape

In [ ]:
# This is the feature vector just created
X_fin

In [ ]:
# build the NLP model using the input and the output, use the Multinomial Naive Bayes
# model to figure out the relationship between the input and the output.
# its a supervised learning algorithm that works really well for text based data
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Instantiate an object model by calling a method MultinomialNB()
model = MultinomialNB()

In [ ]:
# Split the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X_fin, y, test_size = 0.3)

In [ ]:
# Train the model by calling a method fit()
model.fit(X_train,y_train)

In [ ]:
# Call predict() method
# To check how good our model is we first make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report

# Instantiate a mthod named as Cla
cf = classification_report(y_test, y_pred)

# Display the values of an object cf
print(cf)

## Feture Generation using TI-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an object 'tf' by calling a method TfidfVectorizer()
tfidf = TfidfVectorizer(max_features = 3000)

# Train the dataset by calling a method fit_tranform() 
X_tfidf = tfidf.fit_transform(cleaned_data).toarray()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Instantiate an object model by calling a method MultinomialNB()
model_tdidf = MultinomialNB()

In [ ]:
# Split the dataset into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.3)

In [ ]:
# Train the model by calling a method fit()
model_tdidf.fit(X_train, y_train)

In [ ]:
# Call predict() method
y_pred = model_tdidf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

# Instantiate a mthod named as Cla
cf = classification_report(y_test, y_pred)

# Display the values of an object cf
print(cf)

## Time series on milk production.

In [ ]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, SimpleRNN # for creating regular densely-connected NN layers and RNN layers

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import mean_squared_error # for model evaluation metrics
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version